In [1]:
import nbimporter

In [2]:
import utils
import blockchain as blockchain_utils

Importing Jupyter notebook from utils.ipynb
Importing Jupyter notebook from blockchain.ipynb


### LOAD RESOURCES

In [3]:
settings = utils.load_yaml('resources/settings.yaml')

In [4]:
identifier = utils.load_yaml('resources/identifier.yaml')

In [5]:
latest = utils.load_json('resources/latest.json')

### CONNECT TO THE ETHEREUM GATEWAY

In [35]:
web3 = blockchain_utils.connect(settings)

### SERIALIZE THE MANAGER CONTRACT

In [36]:
device_manager = blockchain_utils.contract(latest['device_manager'], web3, settings)

### HASH THE DEVICE IDENTIFIER

In [6]:
hashed_identifier = utils.hash_data(identifier)

### FETCH & SERIALIZE THE DEVICE SPECIFIC CONTRACT

In [11]:
contract = blockchain_utils.contract({
    'address': device_manager.read({
        'func': 'fetch',
        'params': hashed_identifier
    }),
    'abi': latest['device']['abi']
}, web3, settings)

### CALLBACK FUNC FOR UPLOADING DATA

In [27]:
def upload(data):
    print(latest)

### START TRACKING LOG FILE

In [6]:
container = []

In [7]:
utils.track(
    settings['log_path'],
    container
)

LISTENING TO: /var/log/syslog

THE PROCESS WAS MANUALLY KILLED


### GATHERED DATA

In [8]:
container

[{'date': 1612477287,
  'platform': 'desktop',
  'module': 'systemd[968]',
  'msg': 'Started Application launched by gsd-media-keys.'},
 {'date': 1612477287,
  'platform': 'desktop',
  'module': 'systemd[968]',
  'msg': 'Started VTE child process 276982 launched by gnome-terminal-server process 276105.'},
 {'date': 1612477287,
  'platform': 'desktop',
  'module': 'systemd[968]',
  'msg': 'gnome-launched-x-terminal-emulator-276972.scope: Succeeded.'},
 {'date': 1612477299,
  'platform': 'desktop',
  'module': 'systemd[968]',
  'msg': 'Started Application launched by gsd-media-keys.'},
 {'date': 1612477299,
  'platform': 'desktop',
  'module': 'systemd[968]',
  'msg': 'Started VTE child process 277031 launched by gnome-terminal-server process 276105.'},
 {'date': 1612477299,
  'platform': 'desktop',
  'module': 'systemd[968]',
  'msg': 'gnome-launched-x-terminal-emulator-277023.scope: Succeeded.'},
 {'date': 1612477300,
  'platform': 'desktop',
  'module': 'zeitgeist-datah[3414]',
  'msg